In [1]:
from glob import glob
import numpy as np
import pyvox.parser
from sklearn.metrics import accuracy_score, mean_squared_error
from skimage.metrics import hausdorff_distance, structural_similarity
import scipy.spatial.distance as distance
from scipy.spatial.distance import jaccard as jaccard_distance

from metrics import dice


def __read_vox__(path):
        vox = pyvox.parser.VoxParser(path).parse()
        a = vox.to_dense()
        caja = np.zeros((64, 64, 64))
        caja[0:a.shape[0], 0:a.shape[1], 0:a.shape[2]] = a
        return caja

In [2]:
from scipy.spatial import ConvexHull, convex_hull_plot_2d

def __to_index__(vessel_box):
    _list = []
    for i in range(64):
        for j in range(64):
            for k in range(64):
                if vessel_box[i][j][k] == 1:
                    _list.append([i, j, k])
    return np.array(_list)

def __hull_area__(dilated, real):
    hull_dilated = ConvexHull(__to_index__(dilated))
    hull_real = ConvexHull(__to_index__(real))

    return np.abs(hull_dilated.area - hull_real.area)

In [3]:
'''

1 15 40362
15 20 2651
20 30 2285
30 100 1086

'''

'\n\n1 15 40362\n15 20 2651\n20 30 2285\n30 100 1086\n\n'

In [4]:
total_mse = []
total_dice = []
total_hausdorff = []
total_jaccard = []
total_area_hull = []

def calc_metrics(files, name):
    print(name)
    for path_class in files:
        i_class = int(path_class.split('/')[-1])
        class_mse = []
        class_dice = []
        class_hausdorff = []
        class_jaccard = []
        class_area_hull = []
        
        for i_pp in glob(f'{path_class}/*_Dilated.vox'):
            dilated = __read_vox__(i_pp)
            real = __read_vox__(i_pp.replace('Dilated', 'Real'))

            error_mse = mean_squared_error(dilated.reshape(1, -1), real.reshape(1, -1))
            class_mse.append(error_mse)

            error_dice = dice(dilated.reshape(64, 64, 64), real.reshape(64, 64, 64))
            class_dice.append(error_dice)

            error_hausdorff = hausdorff_distance(dilated.reshape(64, 64, 64), real.reshape(64, 64, 64))
            class_hausdorff.append(error_hausdorff)
            
            error_jaccard = jaccard_distance(dilated.reshape(1, -1), real.reshape(1, -1))
            class_jaccard.append(error_jaccard)
            
            area_hull = __hull_area__(dilated.reshape(64, 64, 64), real.reshape(64, 64, 64))
            class_area_hull.append(area_hull)

        print(f'class {i_class} ')
        print('*'*20)
        print(f'      MSE {np.mean(class_mse)} +/- {np.std(class_mse)}')
        print(f'      DICE {np.mean(class_dice)} +/- {np.std(class_dice)}')
        print(f'      HAUSDORFF {np.mean(class_hausdorff)} +/- {np.std(class_hausdorff)}')
        print(f'      JACCARD {np.mean(class_jaccard)} +/- {np.std(class_jaccard)}')
        print(f'      DIFERENCIA DE AREA HULL {np.mean(class_area_hull)} +/- {np.std(class_area_hull)}')
        
        total_mse.append(np.mean(class_mse))
        total_dice.append(np.mean(class_dice))
        total_hausdorff.append(np.mean(class_hausdorff))
        total_jaccard.append(np.mean(class_jaccard))
        total_area_hull.append(np.mean(class_area_hull))
        
    print('*'*20)
    print(f'TOTAL')
    print(f'      MSE {np.mean(total_mse)} +/- {np.std(total_mse)}')
    print(f'      DICE {np.mean(total_dice)} +/- {np.std(total_dice)}')
    print(f'      HAUSDORFF {np.mean(total_hausdorff)} +/- {np.std(total_hausdorff)}')
    print(f'      JACCARD {np.mean(total_jaccard)} +/- {np.std(total_jaccard)}')
    print(f'      DIFERENCIA DE AREA HULL {np.mean(total_area_hull)} +/- {np.std(total_area_hull)}')


    

In [8]:
calc_metrics(sorted(glob('fragmentos_30_100_vis/*')), 'fragmentos_30_100_vis')

fragmentos_30_100_vis
class 1 
********************
      MSE 0.03596663475036621 +/- 0.005408984806498082
      DICE 0.7872900880242282 +/- 0.06688348219703155
      HAUSDORFF 4.165647445094242 +/- 0.5219016796100686
      JACCARD 0.34609562798679083 +/- 0.08546699884042731
      DIFERENCIA DE AREA HULL 428.75303738202564 +/- 122.05041050855296
class 10 
********************
      MSE 0.01714093103846897 +/- 0.00926235931419502
      DICE 0.8418117008874166 +/- 0.06200038076377398
      HAUSDORFF 9.593777558958012 +/- 8.075071638849728
      JACCARD 0.2686035138875013 +/- 0.08545315592073462
      DIFERENCIA DE AREA HULL 249.40210776644938 +/- 704.9016747085873
class 11 
********************
      MSE 0.0180539170356646 +/- 0.00495193148351938
      DICE 0.7599963567757826 +/- 0.08060062749348355
      HAUSDORFF 21.654806648824774 +/- 20.030819567947248
      JACCARD 0.3805912067466905 +/- 0.10071861397333202
      DIFERENCIA DE AREA HULL 1748.2104966434797 +/- 2362.23813968186
class 

In [9]:
calc_metrics(sorted(glob('fragmentos_20_30_vis/*')), 'fragmentos_20_30_vis')

fragmentos_20_30_vis
class 1 
********************
      MSE 0.04113933018275669 +/- 0.012811746437726111
      DICE 0.7420116254613761 +/- 0.09375165022913969
      HAUSDORFF 4.619535161049761 +/- 0.9349762617899313
      JACCARD 0.40176097011762807 +/- 0.11315595540807095
      DIFERENCIA DE AREA HULL 492.75989463605254 +/- 223.41063713320187
class 10 
********************
      MSE 0.016575411331555073 +/- 0.009886052450898727
      DICE 0.844135720871646 +/- 0.06042066480192555
      HAUSDORFF 9.625116085454415 +/- 8.140026591081266
      JACCARD 0.26533992868321554 +/- 0.08352014997987592
      DIFERENCIA DE AREA HULL 207.68192258531738 +/- 622.8968544202673
class 11 
********************
      MSE 0.01879583620557598 +/- 0.006487445854979744
      DICE 0.7477161972693458 +/- 0.09512424879265972
      HAUSDORFF 17.896177308618896 +/- 18.797024895658797
      JACCARD 0.3943759140895703 +/- 0.11312398313472402
      DIFERENCIA DE AREA HULL 1224.5576756492503 +/- 1961.6573250022193
c

In [10]:
calc_metrics(sorted(glob('fragmentos_15_20_vis/*')), 'fragmentos_15_20_vis')

fragmentos_15_20_vis
class 1 
********************
      MSE 0.044270833333333336 +/- 0.012022184149815988
      DICE 0.7357732359480034 +/- 0.08054749854693716
      HAUSDORFF 4.715787105552716 +/- 0.8837193096453139
      JACCARD 0.41224473256094496 +/- 0.0908558128443002
      DIFERENCIA DE AREA HULL 466.0126408491533 +/- 251.0709964601897
class 10 
********************
      MSE 0.018497065168532213 +/- 0.011093041244470149
      DICE 0.8324170991974756 +/- 0.06904268279546243
      HAUSDORFF 8.940138633598426 +/- 7.3307603718982115
      JACCARD 0.28159110566252593 +/- 0.09282133538572329
      DIFERENCIA DE AREA HULL 194.7910508947152 +/- 508.61862716255126
class 11 
********************
      MSE 0.01789108633297926 +/- 0.005542013286197739
      DICE 0.7536006357630275 +/- 0.08174981526434001
      HAUSDORFF 15.863265509810041 +/- 17.21195933667566
      JACCARD 0.38872339007423745 +/- 0.10188894559964641
      DIFERENCIA DE AREA HULL 976.9673034850355 +/- 1651.315536798617
cla